In [3]:
from collections import defaultdict


class Variable:
    def __init__(self, value, local_gradients=[]):
        self.value = value
    
    def __add__(self, other):
        return add(self, other)
    
    def __mul__(self, other):
        return mul(self, other)
    
    def __sub__(self, other):
        return add(self, neg(other))

    def __truediv__(self, other):
        return mul(self, inv(other))
    
def add(a, b):
    value = a.value + b.value    
    local_gradients = (
        (a, 1),
        (b, 1)
    )
    return Variable(value, local_gradients)

def mul(a, b):
    value = a.value * b.value
    local_gradients = (
        (a, b.value),
        (b, a.value)
    )
    return Variable(value, local_gradients)

def neg(a):
    value = -1 * a.value
    local_gradients = (
        (a, -1),
    )
    return Variable(value, local_gradients)

def inv(a):
    value = 1. / a.value
    local_gradients = (
        (a, -1 / a.value**2),
    )
    return Variable(value, local_gradients)


def get_gradients(variable):
    "Compute the first partial derivatives of a variable wrt all the child variables."
    gradients = defaultdict(int)

    def compute_gradients(variable, path_value):
        for child_variable, local_gradient in variable.local_gradients:
            # Multiply all edges of a path
            value_of_path_to_child = path_value * local_gradient
            # Add together the different paths
            gradients[child_variable] += value_of_path_to_child
            # Recurse through the graph
            compute_gradients(child_variable, value_of_path_to_child)

    compute_gradients(variable, path_value=1)
    # path_value=1 is from variable differentiated wrt itself
    return gradients


Example 1

In [ ]:
a = Variable(4)
b = Variable(3)
c = a + b
d = a * c
gradients = get_gradients(d)

print('d.value =', d.value)
print("The partial derivative of d with respect to a =", gradients[a])
print("The partial derivative of d with respect to b =", gradients[b])
print("The partial derivative of d with respect to c =", gradients[c])


Example 2

In [ ]:
def f(a, b):
    return (a / b - a) * (b / a + a + b) * (a - b)

a = Variable(230.3)
b = Variable(33.2)
y = f(a, b)

gradients = get_gradients(y)

print("The partial derivative of y with respect to a =", gradients[a])
print("The partial derivative of y with respect to b =", gradients[b])


A Simple Neural Net

In [ ]:
import numpy as np

# convert NumPy array into array of Variable objects:
to_var = np.vectorize(lambda x : Variable(x))

# get values from array of Variable objects:
to_vals = np.vectorize(lambda variable : variable.value)


In [ ]:
input_size = 50
output_size = 10
lrate = 0.001
x = to_var(np.random.random(input_size))
y_true = to_var(np.random.random(output_size))
weights = to_var(np.random.random((input_size, output_size)))

In [ ]:
import matplotlib.pyplot as plt
np.random.seed(0)

def update_weights(weights, gradients, lrate):
    for _, weight in np.ndenumerate(weights):
        weight.value -= lrate * gradients[weight]

input_size = 50
output_size = 10
lrate = 0.001

x = to_var(np.random.random(input_size))
y_true = to_var(np.random.random(output_size))
weights = to_var(np.random.random((input_size, output_size)))

loss_vals = []
for i in range(100):
    y_pred = x @ weights
    loss = np.sum((y_true - y_pred) * (y_true - y_pred))
    loss_vals.append(loss.value)
    gradients = get_gradients(loss)
    update_weights(weights, gradients, lrate)

plt.plot(loss_vals)
plt.xlabel("Time step")
plt.ylabel("Loss")
plt.title("Single linear layer learning")
plt.show()
